# Rockets, rocket engines - Homework 1

### Imports

In [273]:
import numpy as np
import pandas as pd

### Load datas

In [274]:
stage_df = pd.read_csv('../data/import/data-1.csv')
display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s
0,1,458900,428300,2830
1,2,168300,157300,3140
2,3,50100,46600,3190


In [275]:
payload_df = pd.read_csv('../data/import/data-2.csv')
display(payload_df)
payload_mass = payload_df.loc[payload_df["component"] == "payload", "m_t"].values[0]
print(payload_mass)

,component,m_t
0,payload,22000


22000


## 1. How much Δ𝑣 accelaration can be achived with each stage?

In [276]:
def delta_v(we, m_0, m_f):
     return we * np.log(m_0 / m_f)

In [277]:
stage_df["m_struct_t"] = stage_df["m_0_t"] - stage_df["m_prop_t"]

In [278]:
display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s,m_struct_t
0,1,458900,428300,2830,30600
1,2,168300,157300,3140,11000
2,3,50100,46600,3190,3500


In [279]:
upper_stages_masses = []
for i in range(len(stage_df)):
    upper_stages_masses.append(sum(stage_df.loc[i+1:, "m_0_t"]))

In [280]:
display(upper_stages_masses)

[218400, 50100, 0]

In [281]:
for i in range(len(stage_df)):
    stage_df.loc[i, "m_final_t"] = upper_stages_masses[i] + payload_mass +  stage_df.loc[i, "m_struct_t"]

display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s,m_struct_t,m_final_t
0,1,458900,428300,2830,30600,271000.0
1,2,168300,157300,3140,11000,83100.0
2,3,50100,46600,3190,3500,25500.0


In [282]:
for i in range(len(stage_df)):
    stage_df.loc[i, "delta_v"] = delta_v(stage_df.loc[i, "vel_out_m_s"], stage_df.loc[i, "m_0_t"], stage_df.loc[i, "m_final_t"])

display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s,m_struct_t,m_final_t,delta_v
0,1,458900,428300,2830,30600,271000.0,1490.599206
1,2,168300,157300,3140,11000,83100.0,2215.908674
2,3,50100,46600,3190,3500,25500.0,2154.342753


In [283]:
display(stage_df['delta_v']) # m/s

0    1490.599206
1    2215.908674
2    2154.342753
Name: delta_v, dtype: float64

## 2. Structural coefficient

In [284]:
def structural_coefficient(m_prop, m_initial):
    return m_initial / (m_initial - m_prop)

In [285]:
stage_df["struct_coeff"] = 0.0

In [286]:
for i in range(len(stage_df)):
    stage_df.loc[i, "struct_coeff"] = structural_coefficient(stage_df.loc[i, "m_prop_t"], stage_df.loc[i, "m_0_t"])

In [287]:
display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s,m_struct_t,m_final_t,delta_v,struct_coeff
0,1,458900,428300,2830,30600,271000.0,1490.599206,14.996732
1,2,168300,157300,3140,11000,83100.0,2215.908674,15.300000
2,3,50100,46600,3190,3500,25500.0,2154.342753,14.314286


## 3. Final velocity

In [288]:
delta_v_sum = stage_df["delta_v"].sum()
display(delta_v_sum)

np.float64(5860.8506337930385)

In [289]:
v_earth = 465 # m/s
deg = 45.95

In [290]:
v_final = delta_v_sum + v_earth * np.cos(np.radians(deg))

In [291]:
print(f"Final velocity: {v_final} m/s")
print(f"Final velocity: {v_final / 1000} km/s")

Final velocity: 6184.158553238991 m/s
Final velocity: 6.184158553238991 km/s


## 4. Height

In [292]:
g = 9.81


In [293]:
h = v_final**2 / (2 * g)

In [294]:
print(f"Final height: {h} m")
print(f"Final height: {h / 1000} km")

Final height: 1949226.1473801716 m
Final height: 1949.2261473801716 km


## 5. Single stage

In [295]:
w_s_stage = 3190.0 # m/s

In [296]:
print(f"total velocity: {delta_v_sum} m/s")

total velocity: 5860.8506337930385 m/s


In [297]:
z = np.exp(delta_v_sum / w_s_stage)
display(z)

np.float64(6.279292104874458)

### Initial mass

In [298]:
m_0_s_stage = z/(z-1) * payload_mass
print(f"Initial mass of the first stage: {m_0_s_stage} kg")
print(f"Initial mass of the first stage: {m_0_s_stage / 1000} t")

Initial mass of the first stage: 26167.225370933167 kg
Initial mass of the first stage: 26.167225370933167 t


### Structural mass

In [299]:
m_sum_prop = stage_df["m_prop_t"].sum()

In [300]:
print(f"Total propellant mass: {m_sum_prop} kg")
print(f"Total propellant mass: {m_sum_prop / 1000} t")

Total propellant mass: 632200 kg
Total propellant mass: 632.2 t


In [301]:
m_struct_s_stage = m_0_s_stage - payload_mass - m_sum_prop

In [302]:
print(f"Structural mass of the first stage: {m_struct_s_stage} kg")
print(f"Structural mass of the first stage: {m_struct_s_stage / 1000} t")

Structural mass of the first stage: -628032.7746290668 kg
Structural mass of the first stage: -628.0327746290668 t
